<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/MN_examen1_ejercicio3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

In [2]:
def leer_mtx_diag(A):
    matrix=np.zeros((len(A),len(A)))
    for renglon in A:
        if(len(renglon)>3):
            print("Esto no es diagonal")
        matrix[int(renglon[0]-1)][int(renglon[1]-1)]=float(renglon[2])
    return matrix

In [4]:
def check_diag(A,tol):#Pensado para matrices simetricas
    boolean=True
    breaker=False
    for i in range(len(A)):
        for j in range(i,len(A)):
            if(i!=j):
                if(abs(A[i][j])>tol):
                    boolean=False;
                    breaker=True;
                    break;
        if(breaker==True):
            break
    return(boolean)

In [5]:
txt=np.loadtxt("A_3134.mtx",skiprows=1)
A=leer_mtx_diag(txt)
b=np.loadtxt("b_3134.vec",skiprows=1)
import time
from datetime import timedelta #Para medir tiempos

In [ ]:
#Comparación de métodos.

In [6]:
def diagonal(matriz,b):#Recibe la matriz del sistema.
#Y b representa los valores del sistema a resolver.
  if(type(matriz[0])==type(0.)):
    X=[]
    for i in range(len(matriz)):
      if(matriz[i]==0):
        return('el sistema no tiene solución única')
      else: 
        X.append(b[i]/matriz[i])
    return X
  else:
    X=[]
    for i in range(len(matriz)):
      if(matriz[i][i]==0):
        return('el sistema no tiene solución única')
      else: 
        X.append(b[i]/matriz[i][i])
    return X
print("Metodo 1: Diagonal.")
start = time.monotonic()
print("Nuestra matriz es diagonal: ",check_diag(A,0))#Tolerancia=0
x=diagonal(A,b)
print("solucion exitosa")
print("\nError: ",np.linalg.norm(np.dot(A,x)-b))
end = time.monotonic()
tiempo_total = timedelta(seconds=end - start).total_seconds()
print('Tiempo de ejecución Metodo Diagonal:', tiempo_total, '\n')


Metodo 1: Diagonal.
Nuestra matriz es diagonal:  True
solucion exitosa

Error:  0.0
Tiempo de ejecución Metodo Diagonal: 4.173742 



In [7]:
from numba import njit
@njit  
def factorizar_LU_numba(matriz):
    n=len(matriz)
    L=np.zeros((n,n))
    U=np.zeros((n,n))
    for j in range(n):
        for i in range(j,n):
            sum=0
            for k in range(i):
                sum+=L[i,k]*U[k,j]
            L[i,j]=matriz[i,j]-sum
            
            if(i>j):
                sum=0
                for k in range(j):
                    sum+=L[j,k]*U[k,i]
                U[j,i]=(matriz[j,i]-sum)/L[j,j]
            else: 
                U[i,j]=1 
                if(L[i,i]==0):
                    print("Error en la construcción de L")
    return([L,U])

@njit 
def superior_numba(list,b):
  n=len(list)
  X=np.zeros(n)
  X[n-1]=b[n-1]/list[n-1,n-1]
  for i in range(1,n):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[n-1-j]*list[n-i-1,n-1-j]
    X[n-1-i]=((b[n-1-i]-acum)/list[n-1-i,n-1-i])
    
  return X
@njit
def inferior_numba(list,b):#list debe contener las entradas de la matriz triangular.
#Será aceptable introducir un arreglo de arreglos de distintos tamaños, para solo introducir las entradas no cero.
  X=np.zeros(len(list))
  X[0]=b[0]/list[0][0]
  for i in range(1,len(list)):#len(list) es el número de renglones de la matriz
    acum=0
    for j in range(i):#0<=j<=i-1 
      acum+=X[j]*list[i,j]
    X[i]=((b[i]-acum)/list[i,i])
  return X
def solve(L_m,U_m,b):
    y=inferior_numba(L_m,b)#Resolver Ly=b
    return(superior_numba(U_m,y))#Resolver Ux=y
print("Metodo 2: LU.")
start = time.monotonic()
L,U=factorizar_LU_numba(A)
print("factorizacion exitosa")
x=solve(L_m=L,U_m=U,b=b)
print("solucion exitosa")
print("\nError: ",np.linalg.norm(np.dot(A,x)-b))
end = time.monotonic()
tiempo_total = timedelta(seconds=end - start).total_seconds()
print('Tiempo de ejecución Metodo LU+ Superior+ Inferior:', tiempo_total, '\n')


Metodo 2: LU.
factorizacion exitosa
solucion exitosa

Error:  37279370.63054514
Tiempo de ejecución Metodo LU+ Superior+ Inferior: 135.088655 



In [50]:
import numpy as np
def check_diag_dom(matriz):
    n=len(matriz)
    diago_dom=True
    for i in range(n):
        sum=0
        for j in range(n):
            sum+=matriz[i][j]
        if(2*matriz[i][i]<sum):
            diago_dom=False;
            break;
    return (diago_dom)
def Jacobi(matriz,b,tol=10**(-6),max_iter=100,**x_0,):
    if(not check_diag_dom(matriz)): 
        return("la matriz no es diagonalmente dominante")
    else:
        count=0
        n=len(matriz)
        if(x_0):
            x_0=x_0['x_0']
            x_1=np.zeros(n)
        else:
            x_0=np.zeros(n)
            x_1=np.zeros(n)
        while(count<=max_iter):
            count+=1
            for i in range(n):
                sum=0
                for j in range(n):
                    if(j!=i):
                        sum+=matriz[i][j]*x_0[j]
                x_1[i]=(b[i]-sum)/matriz[i][i]
            if(np.linalg.norm(np.array(x_1)-np.array(x_0))<tol):
                print("numero de iteraciones: ",count)
                return (x_1)
            else:
                x_0=x_1
                x_1=np.zeros(n)
        return ("Iteracion maxima alcanzada")
print("Metodo 3: Jacobi")
start = time.monotonic()
x=Jacobi(A,b=b,tol=10**(-6),max_iter=10000)
print("solucion exitosa")
print("\nError: ",np.linalg.norm(np.dot(A,x)-b))
end = time.monotonic()
tiempo_total = timedelta(seconds=end - start).total_seconds()
print('Tiempo de ejecución Metodo Jacobi:', tiempo_total, '\n')

Metodo 3: Jacobi
numero de iteraciones:  2
solucion exitosa

Error:  0.0
Tiempo de ejecución Metodo Jacobi: 20.246593 



In [52]:
import numpy as np
def check_diag_dom(matriz):
    n=len(matriz)
    diago_dom=True
    for i in range(n):
        sum=0
        for j in range(n):
            sum+=matriz[i][j]
        if(2*matriz[i][i]<sum):
            diago_dom=False;
            break;
    return (diago_dom)
def Gauss_Seidel(matriz,b,tol=10**(-6),max_iter=100,**x_0,):
    if(not check_diag_dom(matriz)): 
        return("la matriz no es diagonalmente dominante")
    else:
        count=0
        n=len(matriz)
        if(x_0):
            x_0=x_0['x_0']
        else:
            x_0=np.zeros(n)
        while(count<=max_iter):
            count+=1
            x_1=np.copy(x_0)
            for i in range(n):
                sum=0
                for j in range(n):
                    if(j!=i):
                        sum+=matriz[i][j]*x_0[j]
                x_0[i]=(b[i]-sum)/matriz[i][i]
            if(np.linalg.norm(np.array(x_1)-np.array(x_0))<tol):
                print("numero de iteraciones: ",count)
                return (x_0)
        return("Iteracion maxima alcanzada")
print("Metodo 4: Gauss-Seidel")
start = time.monotonic()
x=Gauss_Seidel(A,b=b,tol=10**(-6),max_iter=10000)
print("solucion exitosa")
print("\nError: ",np.linalg.norm(np.dot(A,x)-b))
end = time.monotonic()
tiempo_total = timedelta(seconds=end - start).total_seconds()
print('Tiempo de ejecución Metodo Gauss-Seidel', tiempo_total, '\n')

Metodo 4: Gauss-Seidel
numero de iteraciones:  2
solucion exitosa

Error:  0.0
Tiempo de ejecución Metodo Gauss-Seidel 20.236159 

